In [1]:
%cd ../..
%load_ext autoreload
%autoreload 2

/root/capsule/code


In [7]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from pipeline import lab, experiment, ephys, psth_foraging, foraging_analysis

foraging_sessions = (foraging_analysis.SessionTaskProtocol & 'session_task_protocol=100').proj()
import datajoint as dj; dj.conn().connect()

# Add choices and autoregression into linear regression

In [6]:
psth_foraging.UnitPeriodLinearFit.progress()

UnitPeriodLinearFit  Completed 714777 of 714777 (100.0%)   2023-03-29 19:08:31


(0, 714777)

In [15]:
psth_foraging.IndependentVariable()

var_name,desc
choice_ic,ipsi (0) or contra (1)
choice_ic_next,"ipsi (0) or contra (1), next choice"
choice_lr,left (0) or right (1)
contra_action_value,contra action_value
contra_choice_kernel,contra choice_kernel
contra_choice_prob,contra choice_prob
firing_1_back,firing rate of 1 trials back
firing_2_back,firing rate of 2 trials back
firing_3_back,firing rate of 3 trials back
firing_4_back,firing rate of 4 trials back


In [26]:
psth_foraging.LinearModel.load()
psth_foraging.LinearModel()

multi_linear_model,if_intercept Whether intercept is included
"dQ, sumQ, rpe, C*2",1
"dQ, sumQ, rpe, C*2, R*1",1
"dQ, sumQ, rpe, C*2, R*1, t",1
"dQ, sumQ, rpe, C*2, R*5, t",1
"dQ, sumQ, rpe, C*2, t",1
Q_c + Q_i + rpe,1
Q_l + Q_r + rpe,1
Q_rel + Q_tot + rpe,1


## Debug fitting

In [27]:
date, imec, unit = '2021-04-18', 0, 541
unit_key = (ephys.Unit() & (experiment.Session & 'session_date = "2021-04-18"' & 'subject_id = 473361') & {'insertion_number': imec + 1, 'unit_uid': unit}).fetch1("KEY")
key = {**unit_key, 'period': "iti_all", 'multi_linear_model': "dQ, sumQ, rpe, C*2"}    

In [29]:
psth_foraging.UnitPeriodLinearFit.populate(unit_key, display_progress=True)

UnitPeriodLinearFit:  16%|█▌        | 9/56 [00:10<00:54,  1.15s/it]

In [3]:
psth_foraging.UnitPeriodLinearFit()

subject_id institution 6 digit animal ID,session session number,insertion_number,clustering_method,unit,period,"behavior_model If 'best_aic' etc, the best behavioral model for each session; otherwise --> Model.model_id",multi_linear_model,actual_behavior_model,model_r2 r square,model_r2_adj r square adj.,model_p,model_bic,model_aic
